# Stability Fine-Tuning SDK - Dev Test

Thank you for trying the first external beta of the Stability Fine Tuning SDK! Please reach out to us if you have any questions or run into issues using the service. Note that this is a **developer beta** - bugs and quality issues with the generated fine-tunes may occur. Please reach out to Stability if this is the case - and share what you've made as well!

Feel free to implement the gRPC SDK below in your own code, though be warned that the API below is subject to change before public release. A REST API will also be available in the near future.

Known issues:

*  Style fine-tunes may result in overfitting - if this is the case, lower the model strength in the prompt - i.e. the `0.7` in `<model_id:0.7>` within the prompt. You may need to go as low as 0.2 or 0.1.
* We will be exposing test parameters soon - please reach out with examples of datasets that produce overfitting or errors if you have them.
* Current input image limits are 3 minimum for all modes, 128 maximum for style fine-tuning, and 64 maximum for all other modes.

In [ ]:
#@title Install Stability SDK with fine-tuning support
import getpass
import io
import logging
import os
import shutil
import sys
import time
from IPython.display import clear_output
from pathlib import Path
from zipfile import ZipFile

if os.path.exists("../src/stability_sdk"):
    sys.path.append("../src") # use local SDK src
else:
    path = Path('stability-sdk')
    if path.exists():
        shutil.rmtree(path)
        !pip uninstall -y stability-sdk
    !git clone -b "PLATFORM-339" --recurse-submodules https://github.com/Stability-AI/stability-sdk
    !pip install ./stability-sdk

In [ ]:
#@title Connect to the Stability API
from stability_sdk.api import Context, generation
from stability_sdk.finetune import (
    create_model, delete_model, get_model, list_models, resubmit_model, update_model,
    FineTuneMode, FineTuneParameters, FineTuneStatus
)

# @markdown To get your API key visit https://dreamstudio.ai/account. Ensure you are added to the whitelist during external test!
STABILITY_HOST = "grpc-staging.stability.ai:443"
STABILITY_KEY = ""

engine_id = "stable-diffusion-xl-1024-v1-0"

# Create API context to query user info and generate images
context = Context(STABILITY_HOST, STABILITY_KEY, generate_engine_id=engine_id)
(balance, pfp) = context.get_user_info()
print(f"Logged in org:{context._user_organization_id} with balance:{balance}")

# Redirect logs to print statements so we can see them in the notebook
class PrintHandler(logging.Handler):
    def emit(self, record):
        print(self.format(record))
logging.getLogger().addHandler(PrintHandler())
logging.getLogger().setLevel(logging.INFO)

In [ ]:
#@title List fine-tuned models for this user / organization.
models = list_models(context, org_id=context._user_organization_id)
print(f"Found {len(models)} models")
for model in models:
    print(f"  Model {model.id} {model.name} {model.status}")

For training, we need a dataset of images. Please only upload images that you have the permission to use. This can be a folder of images or a .zip file containing your images. Images can be of any aspect ratio, as long as they obey a minimum size of 384px on the shortest side, and a maximum size of 1024px on the longest side. Datasets can range from a minimum of 4 images to a maximum of 128 images.

A larger dataset often tends to result in a more accurate model, but will also take longer to train.

While each mode can accept up to 128 images, we have a few suggestions for a starter dataset based on the mode you are using:



*   Face: 6 or more images.
*   Object: 6 - 10 images.
*   Style: 20 - 30 images.


In [ ]:
#@title Upload ZIP file of images.
training_dir = "./train"
Path(training_dir).mkdir(exist_ok=True)
try:
    from google.colab import files

    upload_res = files.upload()
    extracted_dir = list(upload_res.keys())[0]
    print(f"Received {extracted_dir}")
    if not extracted_dir.endswith(".zip"):
        raise ValueError("Uploaded file must be a zip file")

    zf = ZipFile(io.BytesIO(upload_res[extracted_dir]), "r")
    extracted_dir = Path(extracted_dir).stem
    print(f"Extracting to {extracted_dir}")
    zf.extractall(extracted_dir)

    for root, dirs, files in os.walk(extracted_dir):
        for file in files:

            source_path = os.path.join(root, file)
            target_path = os.path.join(training_dir, file)

            if 'MACOSX' in source_path or 'DS' in source_path:
              continue
            print('Adding input image: ', source_path, target_path)
            # Move the file to the target directory
            shutil.move(source_path, target_path)


except ImportError:
    pass

print(f"Using training images from: {training_dir}")

Now we're ready to train our model. Specify parameters like the name of your model, the training mode, and the guiding prompt for object mode training.

Please note that the training duration will vary based on the size of your dataset, the training mode or the engine that is being fine-tuned on.

However, the following are some rough estimates for the training duration for each mode based on our recommended dataset sizes:

* Face: 4 - 5 minutes.
* Object: 5 - 10 minutes.
* Style: 20 - 30 minutes.


In [ ]:
#@title Perform fine-tuning
model_name = "elliot-dev" #@param {type:"string"}
#@markdown > Model names are unique, and may only contain numbers, letters, and hyphens.
training_mode = "face" #@param ["face", "style", "object"] {type:"string"}
#@markdown > The Face training_mode expects pictures containing a face, and automatically crops and centers on the face detected in the input photos. Object segments out the object specified with the prompt below; and Style simply crops the images and filters for image quality.
object_prompt = "cat" #@param {type:"string"}
#@markdown > The Object Prompt is used for segmenting out your subject in the Object fine tuning mode - i.e. if you want to fine tune on a cat, put `cat` - for a bottle of liquor, use `bottle`. In general, it's best to use the most general word you can to describe your object.

print(training_dir)
print(len(os.listdir(training_dir)))
# Gather training images
images = []
for filename in os.listdir(training_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        images.append(os.path.join(training_dir, filename))

# Create the fine-tune model
params = FineTuneParameters(
    name=model_name,
    mode=FineTuneMode(training_mode),
    object_prompt=object_prompt,
    engine_id=engine_id,
)
model = create_model(context, params, images)
print(f"Model {model_name} created.")
print(model)

In [ ]:
#@title Check on training status
start_time = time.time()
while model.status != FineTuneStatus.COMPLETED and model.status != FineTuneStatus.FAILED:
    model = get_model(context, model.id)
    elapsed = time.time() - start_time
    clear_output(wait=True)
    print(f"Model {model.name} ({model.id}) status: {model.status} for {elapsed:.0f} seconds")
    time.sleep(5)

clear_output(wait=True)
status_message = "completed" if model.status == FineTuneStatus.COMPLETED else "failed"
print(f"Model {model.name} ({model.id}) {status_message} after {elapsed:.0f} seconds")

In [ ]:
#@title If fine-tuning fails for some reason, you can resubmit the model
if model.status == FineTuneStatus.FAILED:
    print("Training failed, resubmitting")
    model = resubmit_model(context, model.id)

In [ ]:
#@title <font color="#FFFFFF">9. Generate images from your fine-tuned model
results = context.generate(
    prompts=[f"Illustration of <{model.id}:1> as a wizard"],
    weights=[1],
    width=1024,
    height=1024,
    seed=42,
    steps=40,
    sampler=generation.SAMPLER_DDIM,
    preset="photographic",
)
image = results[generation.ARTIFACT_IMAGE][0]
display(image)

In [ ]:
#@title Models can be updated to change settings before a resubmit or after training to rename
update_model(context, model.id, name="cat-ft-01-renamed")

In [ ]:
#@title Delete the model when it's no longer needed
delete_model(context, model.id)